### Restart the failed us-east import

In [ ]:
!import.sh -o import -t 24 -l import/statements_live-us-east -json {\"waitMessageTimeoutSeconds\":300} -r us-east -b / -a -w 400000 -bulk > import-useast-live-statements-part3.log

After having started the import we can see that the number of threads decreases: 

 - first to 16: this makes sense since 8 partitions are empty
 - then to 8!!!

Running stream.sh

    > stream.sh lag -k --verbose -l import/statements_live-us-east
    
| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|120717538|424811048|545528586|14069052|22937322|
|0|8728788|14208534|22937322|14208534|22937322|
|1|8731460|14149276|22880736|14149276|22880736|
|2|8710766|14197138|22907904|14197138|22907904|
|3|8723144|14176024|22899168|14176024|22899168|
|4|8726814|14203752|22930566|14203752|22930566|
|5|8797547|14131063|22928610|14131063|22928610|
|6|8659158|14186244|22845402|14186244|22845402|
|7|8768832|14139036|22907868|14139036|22907868|
|8|8503380|14126196|22629576|14126196|22629576|
|9|8587516|14070506|22658022|14070506|22658022|
|10|8527171|14135177|22662348|14135177|22662348|
|11|8571420|14069052|22640472|14069052|22640472|
|12|4188615|18438921|22627536|18438921|22627536|
|13|4251018|18382104|22633122|18382104|22633122|
|14|4200629|18437197|22637826|18437197|22637826|
|15|4041280|18630620|22671900|18630620|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|
    

In the logs I can see a lot of rebalance:
    
    Rollback current batch because of consumer rebalancing
    
    Incomplete rebalance during poll, raising exception, revoked: true, lost: false

Looking at datadog the number of threads decreases regularly and the throughput also goes down:

<img src="monitoring/import-us-east-threads.png"/>

Doing one more stream.sh lag -k

| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|115319538|430209048|545528586|14385480|22937322|
|0|8412344|14524978|22937322|14524978|22937322|
|1|8403451|14477285|22880736|14477285|22880736|
|2|8391416|14516488|22907904|14516488|22907904|
|3|8393604|14505564|22899168|14505564|22899168|
|4|8399592|14530974|22930566|14530974|22930566|
|5|8472783|14455827|22928610|14455827|22928610|
|6|8334624|14510778|22845402|14510778|22845402|
|7|8442882|14464986|22907868|14464986|22907868|
|8|8176716|14452860|22629576|14452860|22629576|
|9|8272542|14385480|22658022|14385480|22658022|
|10|8202332|14460016|22662348|14460016|22662348|
|11|8247576|14392896|22640472|14392896|22640472|
|12|3820467|18807069|22627536|18807069|22627536|
|13|3886918|18746204|22633122|18746204|22633122|
|14|3843078|18794748|22637826|18794748|22637826|
|15|3619213|19052687|22671900|19052687|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|

So, it seems that there was progres on each of the partitions ...